In [1]:
# Importing packages
import requests
import json
import time #will use this to pause execution for a few seconds

import numpy as np
import pandas as pd

from pandas import json_normalize

In [10]:
# Extracting Google Key
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

# extract key from a separate ini file where the key is stored
API_KEY = config['google']['api_key']

In [3]:
# The initial string for the api reuqest
endpoint_url_start = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [4]:
# Creating dataframes to be filled
hilo_common = pd.DataFrame(columns=['name','place_id', 'address','coordinates',\
                                    'business_status', 'types', 'delivery', 'dine_in',\
                                    'takeout', 'price_level', 'rating', 'user_ratings_total',\
                                    'serves_breakfast', 'serves_brunch','serves_lunch',\
                                    'serves_dinner', 'serves_vegetarian_food', 'serves_beer',
                                    'serves_wine'])
hilo_uncommon = pd.DataFrame(columns=['name','place_id', 'address','coordinates',\
                                    'business_status', 'types', 'delivery', 'dine_in',\
                                    'takeout', 'price_level', 'rating', 'user_ratings_total',\
                                    'serves_breakfast', 'serves_brunch','serves_lunch',\
                                    'serves_dinner', 'serves_vegetarian_food', 'serves_beer',
                                    'serves_wine'])

In [5]:
# Specifying the types of places in lists
# common = multiple small radii for api search
# uncommon = whole city radius search
places_common = ['bakery', 'cafe', 'restaurant', 'meal_delivery', 'meal_takeaway', 'bar', 'convenience store',\
                 'gas_station', 'supermarket', 'lodging', 'department_store', 'tourist_attraction']
places_uncommon = ['airport', 'amusement_park', 'aquarium', 'casino', 'hospital',\
                   'liquor_store', 'stadium', 'university', 'zoo', 'movie_theater', 'primary_school',\
                   'school', 'secondary_school', 'shopping mall', 'bowling_alley']

In [6]:
# uncommon/large radius search info
central_cord = "19.68933, -155.08268"
central_radius = "9500"

In [7]:
# common/small radius searches info
cords = ["19.71890,-155.09343", "19.72473,-155.06600", "19.72600,-155.02977", "19.70562,-155.08258",\
         "19.70661,-155.05612", "19.69671,-155.11744", "19.67937,-155.07814", "19.67890,-155.03994",\
         "19.68336,-155.16312", "19.64372,-155.11469"]
radii = ["1500", "1500", "2500", "1500",\
         "1500", "2500", "2500", "3000",\
         "3000", "4000"]

In [8]:
# function to append result to the dataframe
# Takes care of null returned from API request
def append_place(json_results, df):
    l = len(df)
    for i, val in enumerate(json_results['results']):
        df.loc[l + i] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         val.get('vicinity', np.nan), 
                         val.get('geometry', np.nan).get('location', np.nan),
                         val.get('business_status', np.nan),
                         val.get('types', np.nan),
                         val.get('delivery', np.nan),
                         val.get('dine_in', np.nan),
                         val.get('takeout', np.nan),
                         val.get('price_level', np.nan),
                         val.get('rating', np.nan),
                         val.get('user_ratings_total', np.nan),
                         val.get('serves_breakfast', np.nan),
                         val.get('serves_brunch', np.nan),
                         val.get('serves_lunch',np.nan),
                         val.get('serves_dinner', np.nan),
                         val.get('serves_vegetarian_food', np.nan),
                         val.get('serves_beer', np.nan),
                         val.get('serves_wine', np.nan)]

In [13]:
# creating a dataset for the uncommon places
hilo_uncommon_full = pd.DataFrame(columns=['name','place_id', 'address','coordinates',\
                                    'business_status', 'types', 'delivery', 'dine_in',\
                                    'takeout', 'price_level', 'rating', 'user_ratings_total',\
                                    'serves_breakfast', 'serves_brunch','serves_lunch',\
                                    'serves_dinner', 'serves_vegetarian_food', 'serves_beer',
                                    'serves_wine'])
for i in range(5):
    for _type in places_uncommon:
        endpoint_url = endpoint_url_start + "&location=" + central_cord + "&radius=" + \
        central_radius + "&region=us&type=" + _type + "&key=" + API_KEY
        
        params = {}
        
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        append_place(results, hilo_uncommon)
        
        
        time.sleep(1)
        
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token']
            res = requests.get(endpoint_url, params = params)
            results =  json.loads(res.content)
            append_place(results, hilo_uncommon)
            time.sleep(1)
    hilo_uncommon_full = pd.concat([hilo_uncommon_full, hilo_uncommon])

In [16]:
# drop duplicates and reset index
hilo_uncommon_full.drop_duplicates(subset=['place_id'], inplace=True)
hilo_uncommon_full.reset_index(drop=True, inplace=True)
hilo_uncommon_full['category'] = hilo_uncommon_full['types'].apply(lambda x: x[0])
hilo_uncommon_full.head()


,name,place_id,address,coordinates,business_status,types,delivery,dine_in,takeout,price_level,rating,user_ratings_total,serves_breakfast,serves_brunch,serves_lunch,serves_dinner,serves_vegetarian_food,serves_beer,serves_wine,category
0,Hilo International Airport,ChIJA_6Mr3hMUnkRf2xxZUX-my4,"2450 Kekuanaoa Street, Hilo","{'lat': 19.7188342, 'lng': -155.0416866}",OPERATIONAL,"[airport, point_of_interest, establishment]",NaN,NaN,NaN,NaN,4.4,819.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,airport
1,Blue Hawaiian Helicopters,ChIJqx06aMK0U3kR6i4j-j9NX34,United States,"{'lat': 19.7150176, 'lng': -155.0434895}",OPERATIONAL,"[airport, travel_agency, tourist_attraction, p...",NaN,NaN,NaN,NaN,4.4,320.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,airport
2,Hilo Medical Center Heliport,ChIJeQfmUem0U3kR8UKXpxmqQfY,"1190 Waianuenue Avenue, Hilo","{'lat': 19.7184778, 'lng': -155.1143653}",OPERATIONAL,"[airport, point_of_interest, establishment]",NaN,NaN,NaN,NaN,4.5,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,airport
3,ヒロ国際空港,ChIJnTPqSw9NUnkRfbssL0BYEho,Hilo,"{'lat': 19.7214152, 'lng': -155.0417123}",OPERATIONAL,"[airport, point_of_interest, establishment]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,airport
4,Hilo Medical Center,ChIJ6VYwrKS0U3kR9caoLq5TmrM,"1190 Waianuenue Avenue, Hilo","{'lat': 19.7182136, 'lng': -155.11293}",OPERATIONAL,"[hospital, health, point_of_interest, establis...",NaN,NaN,NaN,NaN,3.3,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital


In [19]:
# creating a dataset for the common places
for idx, circle in enumerate(cords):
    for _type in places_common:
        endpoint_url = endpoint_url_start + "&location=" + circle + "&radius=" + \
        radii[idx] + "&region=us&type=" + _type + "&key=" + API_KEY

        params = {}

        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        append_place(results, hilo_common)

        time.sleep(1)
        
        # Pulling results from other pages
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token']
            res = requests.get(endpoint_url, params = params)
            results =  json.loads(res.content)
            append_place(results, hilo_common)
            time.sleep(1)

In [22]:
# drop duplicates and reset index
hilo_common.drop_duplicates(subset=['place_id'], inplace=True)
hilo_common.reset_index(drop=True, inplace=True)
hilo_common['category'] = hilo_common['types'].apply(lambda x: x[0])
hilo_common.head()

,name,place_id,address,coordinates,business_status,types,delivery,dine_in,takeout,price_level,rating,user_ratings_total,serves_breakfast,serves_brunch,serves_lunch,serves_dinner,serves_vegetarian_food,serves_beer,serves_wine,category
0,Papa'a Palaoa Bakery,ChIJV30QdVdLUnkRF1oH6Bho5SU,"187 Kilauea Avenue, Hilo","{'lat': 19.7220616, 'lng': -155.085246}",OPERATIONAL,"[bakery, store, point_of_interest, food, estab...",NaN,NaN,NaN,1.0,4.7,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bakery
1,Low International Food,ChIJzRUOcldLUnkRuVdgapXhlEg,"222 Kilauea Avenue, Hilo","{'lat': 19.7216053, 'lng': -155.0851868}",OPERATIONAL,"[bakery, store, restaurant, point_of_interest,...",NaN,NaN,NaN,1.0,4.3,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bakery
2,Moonstruck,ChIJB-pawVdLUnkR8krKiIkAgus,"16 Furneaux Lane, Hilo","{'lat': 19.7240477, 'lng': -155.0857916}",OPERATIONAL,"[bakery, store, point_of_interest, food, estab...",NaN,NaN,NaN,2.0,4.5,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bakery
3,Nector Cafe,ChIJj2hxXbdLUnkRftSaRFVgczg,"216 Kamehameha Avenue, Hilo","{'lat': 19.7248671, 'lng': -155.086052}",OPERATIONAL,"[bakery, cafe, store, point_of_interest, food,...",NaN,NaN,NaN,NaN,4.1,159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bakery
4,Grandma's Cookie Jar,ChIJZcKu36tMUnkR3tp-0kDxS-Y,Hilo,"{'lat': 19.7297152, 'lng': -155.0900154}",OPERATIONAL,"[bakery, store, point_of_interest, food, estab...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bakery


In [23]:
# creating a list of unique place ids that we can use in Google Places Details API search to get business hours
h_common_ids = np.unique(hilo_common['place_id'])
h_uncommon_ids = np.unique(hilo_uncommon_full['place_id'])

In [24]:
# creating dataset to fill with business hours
h_common_hours = pd.DataFrame(columns=['Name','place_id','business_hours', 'business_hours_text'])
h_uncommon_hours = pd.DataFrame(columns=['Name','place_id','business_hours','business_hours_text'])

In [25]:
# new api request string for new type of API requests
endpoint_url_start_2 = "https://maps.googleapis.com/maps/api/place/details/json?place_id="

In [26]:
# new function to append results from Places Details search to business hours dataset
def append_place_2(json_results, df):
    l = len(df)
    val = json_results['result']
    if len(val) == 2:
        df.loc[l + 1] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         np.nan,
                         np.nan]
    else:
        df.loc[l + 1] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         val.get('opening_hours', np.nan).get('periods', np.nan),
                         val.get('opening_hours', np.nan).get('weekday_text', np.nan)]

In [27]:
# creating a business hours dataset for the common places
for place_id in h_common_ids:
    endpoint_url = endpoint_url_start_2 + place_id + \
    "&fields=name%2Cplace_id%2Copening_hours&key=" + API_KEY
    
    params = {}
    
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    append_place_2(results, h_common_hours)
    
    time.sleep(1)

In [31]:
# creating a business hours dataset for the uncommon places
for place_id in h_uncommon_ids:
    endpoint_url = endpoint_url_start_2 + place_id + \
    "&fields=name%2Cplace_id%2Copening_hours&key=" + API_KEY
    
    params = {}
    
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    append_place_2(results, h_uncommon_hours)
    
    
    time.sleep(1)

In [40]:
# merging previous datasets with business hours datasets
hilo_common = hilo_common.merge(h_common_hours, how='left', left_on=['name', 'place_id'], right_on=['Name', 'place_id'])
hilo_uncommon_full = hilo_uncommon_full.merge(h_uncommon_hours, how='left', left_on=['name', 'place_id'], right_on=['Name', 'place_id'])

In [41]:
# exporting data sets to local csv files
hilo_common.to_csv('hilo_common.csv')
hilo_uncommon_full.to_csv('hilo_uncommon.csv')